<a href="https://colab.research.google.com/github/MajesticFT/SuperAI2/blob/main/Week11_SmartMaintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!gdown --id 1YrFt22iuc_Oujg46D1-DHOX3Xs0gZJml

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1YrFt22iuc_Oujg46D1-DHOX3Xs0gZJml 



In [ ]:
!wget https://learn.aiat.or.th/download/ryxderyseyh3535/train_data1200.zip

--2022-04-08 08:27:18--  https://learn.aiat.or.th/download/ryxderyseyh3535/train_data1200.zip
Resolving learn.aiat.or.th (learn.aiat.or.th)... 110.78.211.34
Connecting to learn.aiat.or.th (learn.aiat.or.th)|110.78.211.34|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-04-08 08:27:19 ERROR 404: Not Found.



In [ ]:
!unzip -q '/content/train_data600.zip'

unzip:  cannot find or open /content/train_data600.zip, /content/train_data600.zip.zip or /content/train_data600.zip.ZIP.


In [ ]:
!unzip -q '/content/train_data1200.zip'

unzip:  cannot find or open /content/train_data1200.zip, /content/train_data1200.zip.zip or /content/train_data1200.zip.ZIP.


In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from tqdm import tqdm
from scipy.fft import rfft, rfftfreq
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.signal import wiener, hilbert

In [ ]:
count = 0
for files in os.listdir('/content/drive/Shareddrives/Hackathon_Week11_Smart_Maintenance/train_data600'):
  count+=1
count

29001

In [ ]:
label_600 = pd.read_csv("/content/drive/Shareddrives/Hackathon_Week11_Smart_Maintenance/label600.csv", header=None)

In [ ]:
label_600

,0,1
0,train_data1.csv,10
1,train_data2.csv,26
2,train_data3.csv,14
3,train_data4.csv,4
4,train_data5.csv,8
...,...,...
28995,train_data28996.csv,14
28996,train_data28997.csv,24
28997,train_data28998.csv,9
28998,train_data28999.csv,19


In [ ]:
label_600[label_600[1]==0]

Functions for converting the time series to frequency domain by using FFT and applying Wiener Filter

In [ ]:
def getFeature(sinewave):
    sample_rate = 51200
    start_time = 0
    end_time = 0.25
    time = np.arange(start_time, end_time, 1/sample_rate)

    freq = fft(sinewave)/len(time)
    hz = np.linspace(0, sample_rate/2, int(np.floor(len(time)/2)+1))
    y=2 * np.abs(freq)
    filtered_img = wiener(y)
    tmp = filtered_img
    return tmp[1:2001]
def getFeature3channel(data):
    return np.array(list(map(getFeature, data)))

def getFeatureDataset(dataset):
    return np.array(list(map(getFeature3channel, dataset)))

Convert csv to ndarray and apply FFT and Wiener filter

In [ ]:
def preprocessing(directory, training=True):
  x_array = []
  y_array = []
  file_list = []
  for filename in tqdm(os.listdir(directory)):
    if filename != "label.csv" and filename.endswith('.csv'):
      df = pd.read_csv(os.path.join(directory, filename), header=None)
      arr = df.to_numpy()
      #arr_t = np.swapaxes(arr, 0, 1)
      #ffted=[]
      #for i in arr_t:

        # dt = 0.25/12500
        # t = np.arange(0,0.25,dt)
        # n = len(t)
        # L = np.arange(1,np.floor(n/2),dtype='int')
        # fhat = np.fft.rfft(i,n)
        # PSD = fhat*np.conj(fhat) / n
        # freq = (1/(dt*n)) * np.arange(n)
        # indices = PSD > 3
        # PSDclean = PSD * indices
        # fhat = indices * fhat
        # ffilt = np.fft.ifft(fhat)
        # ffted.append(np.real(PSD[L][0:200]))
        #ffted.append(X_train)
      #ffted = np.asarray(ffted)
      x_array.append(arr)
     
      if training:
        label = label_600[label_600[0]==filename][1]
        y_array.append(label)
      else: 
        file_list.append(filename)
  
  
  x_array = getFeatureDataset(np.swapaxes(np.asarray(x_array), 1, 2))
  x_array = np.expand_dims(x_array, axis=1)
  if training:
    y_array = np.asarray(y_array)
    return x_array, y_array
  else: return x_array, file_list


In [ ]:
x_array_600, y_array_600 = preprocessing("/content/drive/Shareddrives/Hackathon_Week11_Smart_Maintenance/train_data600")
x_array_600

  0%|          | 6/29029 [00:07<10:32:48,  1.31s/it]


KeyboardInterrupt: ignored

In [ ]:
x_array_600.shape

(29000, 1, 3, 2000)

# Trying to convert labels to 3 catagories (although it didn't work)

In [ ]:
def to3Labels(y_array):
  size = y_array.shape[0]
  y_array_ub = []
  y_array_ma = []
  y_array_mp = []
  for i in range(size):

    if y_array[i] in [0, 4, 5, 6, 7, 8, 9, 10]:
        y_array_ub.append(0)
    elif y_array[i] in [1, 11, 14, 17, 20, 23, 26]:  
        y_array_ub.append(1)
    elif y_array[i] in [2, 12, 15, 18, 21, 24, 27]:  
        y_array_ub.append(2)
    else:  y_array_ub.append(3)

    if y_array[i] in [0, 1, 2, 3, 7, 8, 9, 10, 20, 21, 22, 23, 24, 25, 26, 27, 28]:
        y_array_ma.append(0)
    elif y_array[i] in [4, 11, 12, 13]:  
        y_array_ma.append(1)
    elif y_array[i] in [5, 14, 15, 16]:  
        y_array_ma.append(2)
    else: y_array_ma.append(3)

    if y_array[i] in [0, 1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19]:
        y_array_mp.append(0)
    elif y_array[i] in [7, 20, 21, 22]:  
        y_array_mp.append(1)
    elif y_array[i] in [8, 23, 24, 25]:  
        y_array_mp.append(2)
    elif y_array[i] in [9, 26, 27, 28]:  
        y_array_mp.append(3)
    else: y_array_mp.append(4)
  y_array_ub = np.asarray(y_array_ub)
  y_array_ma = np.asarray(y_array_ma)
  y_array_mp = np.asarray(y_array_mp)
  return y_array_ub, y_array_ma, y_array_mp


# Combined with 1200rpm dataset

In [ ]:
x_array_1200, y_array_1200 = preprocessing('/content/train_data1200')

100%|██████████| 29001/29001 [08:13<00:00, 58.75it/s]


In [ ]:
x_array = np.concatenate((x_array_600, x_array_1200))
y_array = np.concatenate((y_array_600, y_array_1200))

In [ ]:
x_array.shape

(58000, 1, 3, 500)

In [ ]:
y_array.shape

(58000, 1)

# Models

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.25, stratify= y_array, random_state=112)

In [ ]:
y_train.shape

(43500, 1)

In [ ]:
#if doing 3 label categories
y_train_ub, y_train_ma, y_train_mp = to3Labels(y_train)
y_test_ub, y_test_ma, y_test_mp = to3Labels(y_test)

In [ ]:
x_train.shape

(43500, 1, 3, 2000)

In [ ]:
y_train_ub.shape

(43500,)

In [ ]:
!pip install min2net

     |████████████████████████████████| 58 kB 4.1 MB/s 
     |████████████████████████████████| 1.0 MB 17.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=2349e6c3ee284ae60063144784e63b715efd997c67965db28dd43bd02eabe0d2
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
from min2net.model import EEGNet, DeepConvNet

In [ ]:
model = DeepConvNet(input_shape=(1,3,2000),
                    num_class=29,
                    dropout_rate=0.2,
                    patience = 8,
                    epochs = 300,
                    factor = 0.5,
                    lr = 1e-3,
                    min_lr = 1e-6,
                    es_patience = 30,
                    batch_size = 256,
                    shuffle=True
                    )
model.fit(x_train, y_train, x_test, y_test)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1, 3, 2000)]      0         
                                                                 
 conv2d_15 (Conv2D)          (None, 8, 3, 2000)        1600      
                                                                 
 batch_normalization_12 (Bat  (None, 8, 3, 2000)       8000      
 chNormalization)                                                
                                                                 
 depthwise_conv2d (Depthwise  (None, 16, 1, 2000)      48        
 Conv2D)                                                         
                                                                 
 batch_normalization_13 (Bat  (None, 16, 1, 2000)      8000      
 chNormalization)                                                
                                                           

In [ ]:
#if doing 3 label categories
model_ub = DeepConvNet(input_shape=(1,3,2000),
                    num_class=4,
                    dropout_rate=0,
                    patience = 10,
                    epochs = 300,
                    factor = 0.5,
                    lr = 1e-3,
                    min_lr = 1e-6,
                    es_patience = 25,
                    batch_size = 256,
                    shuffle=True
                    )
model_ub.fit(x_train, y_train_ub, x_test, y_test_ub)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1, 3, 2000)]      0         
                                                                 
 conv2d_25 (Conv2D)          (None, 25, 3, 1996)       150       
                                                                 
 conv2d_26 (Conv2D)          (None, 25, 1, 1996)       1900      
                                                                 
 batch_normalization_20 (Bat  (None, 25, 1, 1996)      100       
 chNormalization)                                                
                                                                 
 activation_25 (Activation)  (None, 25, 1, 1996)       0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 25, 1, 998)       0         
 g2D)                                                      

In [ ]:
#if doing 3 label categories
model_ma = DeepConvNet(input_shape=(1,3,2000),
                    num_class=4,
                    dropout_rate=0,
                    patience = 10,
                    epochs = 300,
                    factor = 0.5,
                    lr = 1e-3,
                    min_lr = 1e-6,
                    es_patience = 25,
                    batch_size = 256,
                    shuffle=True
                    )
model_ma.fit(x_train, y_train_ma, x_test, y_test_ma)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1, 3, 2000)]      0         
                                                                 
 conv2d_35 (Conv2D)          (None, 25, 3, 1996)       150       
                                                                 
 conv2d_36 (Conv2D)          (None, 25, 1, 1996)       1900      
                                                                 
 batch_normalization_28 (Bat  (None, 25, 1, 1996)      100       
 chNormalization)                                                
                                                                 
 activation_35 (Activation)  (None, 25, 1, 1996)       0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 25, 1, 998)       0         
 g2D)                                                      

In [ ]:
#if doing 3 label categories
model_mp = DeepConvNet(input_shape=(1,3,2000),
                    num_class=5,
                    dropout_rate=0,
                    patience = 10,
                    epochs = 300,
                    factor = 0.5,
                    lr = 1e-3,
                    min_lr = 1e-6,
                    es_patience = 25,
                    batch_size = 256,
                    shuffle=True
                    )
model_mp.fit(x_train, y_train_mp, x_test, y_test_mp)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 3, 2000)]      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 25, 3, 1996)       150       
                                                                 
 conv2d_11 (Conv2D)          (None, 25, 1, 1996)       1900      
                                                                 
 batch_normalization_8 (Batc  (None, 25, 1, 1996)      100       
 hNormalization)                                                 
                                                                 
 activation_10 (Activation)  (None, 25, 1, 1996)       0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 25, 1, 998)       0         
 2D)                                                       

# Predictions

In [ ]:
!wget https://learn.aiat.or.th/download/rgh4h5/final_test_dataset.zip

--2022-04-07 13:07:17--  https://learn.aiat.or.th/download/rgh4h5/final_test_dataset.zip
Resolving learn.aiat.or.th (learn.aiat.or.th)... 110.78.211.34
Connecting to learn.aiat.or.th (learn.aiat.or.th)|110.78.211.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 562054701 (536M) [application/zip]
Saving to: ‘final_test_dataset.zip’

final_test_dataset. 100%[===================>] 536.02M  15.9MB/s    in 37s     

2022-04-07 13:07:55 (14.6 MB/s) - ‘final_test_dataset.zip’ saved [562054701/562054701]



In [ ]:
os.makedirs('/content/test_dataset')

In [ ]:
!unzip /content/final_test_dataset.zip -d /content/test_dataset

Archive:  /content/final_test_dataset.zip
   creating: /content/test_dataset/final_125000_test_dataset/
  inflating: /content/test_dataset/final_125000_test_dataset/test_data0.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data10.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data100.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1000.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1001.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1002.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1003.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1004.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1005.csv  
  inflating: /content/test_dataset/final_125000_test_dataset/test_data1006.csv  
  inflating: /

In [ ]:
count=0
for filename in os.listdir('/content/test_dataset'):
  #if filename.endswith('.csv'):
    count+=1
count

9001

In [ ]:
test_preprocessed, file_list = preprocessing('/content/test_dataset/final_125000_test_dataset', False)
test_preprocessed

100%|██████████| 1800/1800 [00:16<00:00, 107.21it/s]


array([[[[0.00383867, 0.01327733, 0.02578502, ..., 0.05492069,
          0.05364118, 0.03558253],
         [0.00430348, 0.01015933, 0.01290986, ..., 0.01448439,
          0.03051759, 0.0130348 ],
         [0.00783522, 0.0172916 , 0.03496742, ..., 0.04422406,
          0.0380878 , 0.03940645]]],


       [[[0.00286146, 0.00818032, 0.00959736, ..., 0.01927995,
          0.02927332, 0.01633788],
         [0.00411028, 0.00732311, 0.00844265, ..., 0.00409388,
          0.02514697, 0.01253675],
         [0.00515118, 0.01359107, 0.01168747, ..., 0.03552276,
          0.04347438, 0.02475249]]],


       [[[0.00951685, 0.01880737, 0.02492013, ..., 0.06219242,
          0.07574457, 0.07466477],
         [0.00143503, 0.00734462, 0.01257323, ..., 0.02315752,
          0.02552488, 0.01997022],
         [0.01255551, 0.0550151 , 0.06428   , ..., 0.05799996,
          0.0813975 , 0.07216995]]],


       ...,


       [[[0.00587882, 0.00807228, 0.02209996, ..., 0.02436702,
          0.04976074, 0.04765

In [ ]:
#test_preprocessed = np.expand_dims(test_preprocessed, axis=1)

In [ ]:
test_preprocessed.shape

(1800, 1, 3, 2000)

In [ ]:
file_list

['test_data795.csv',
 'test_data1110.csv',
 'test_data1363.csv',
 'test_data769.csv',
 'test_data1442.csv',
 'test_data1425.csv',
 'test_data277.csv',
 'test_data1165.csv',
 'test_data345.csv',
 'test_data1392.csv',
 'test_data197.csv',
 'test_data896.csv',
 'test_data172.csv',
 'test_data951.csv',
 'test_data786.csv',
 'test_data1028.csv',
 'test_data1320.csv',
 'test_data1691.csv',
 'test_data676.csv',
 'test_data625.csv',
 'test_data1495.csv',
 'test_data414.csv',
 'test_data517.csv',
 'test_data645.csv',
 'test_data1043.csv',
 'test_data1637.csv',
 'test_data408.csv',
 'test_data655.csv',
 'test_data1724.csv',
 'test_data1535.csv',
 'test_data382.csv',
 'test_data1465.csv',
 'test_data688.csv',
 'test_data1711.csv',
 'test_data1595.csv',
 'test_data1047.csv',
 'test_data1423.csv',
 'test_data1548.csv',
 'test_data12.csv',
 'test_data1745.csv',
 'test_data623.csv',
 'test_data349.csv',
 'test_data921.csv',
 'test_data1104.csv',
 'test_data681.csv',
 'test_data929.csv',
 'test_data75

In [ ]:
blanks = np.zeros(1800)

In [ ]:
predictions, eval = model.predict(test_preprocessed, blanks)

8/8 [==============================] - 0s 12ms/step - loss: 8.8866 - accuracy: 0.0583
              precision    recall  f1-score   support

         0.0       1.00      0.06      0.11      1800
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00         0
         3.0       0.00      0.00      0.00         0
         4.0       0.00      0.00      0.00         0
         5.0       0.00      0.00      0.00         0
         6.0       0.00      0.00      0.00         0
         7.0       0.00      0.00      0.00         0
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0
        10.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        12.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0
        14.0       0.00      0.00      0.00         0
        15.0       0.00      0.00      0.00      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#if doing 3 label categories
predictions_ub, eval_ub = model_ub.predict(test_preprocessed, blanks)

36/36 [==============================] - 1s 13ms/step - loss: 4.4963 - accuracy: 0.2950
              precision    recall  f1-score   support

         0.0       1.00      0.29      0.46      9000
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00         0
         3.0       0.00      0.00      0.00         0

    accuracy                           0.29      9000
   macro avg       0.25      0.07      0.11      9000
weighted avg       1.00      0.29      0.46      9000

F1-score is computed based on macro


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#if doing 3 label categories
predictions_ma, eval_ma = model_ma.predict(test_preprocessed, blanks)

36/36 [==============================] - 1s 13ms/step - loss: 3.9791 - accuracy: 0.5869
              precision    recall  f1-score   support

         0.0       1.00      0.59      0.74      9000
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00         0
         3.0       0.00      0.00      0.00         0

    accuracy                           0.59      9000
   macro avg       0.25      0.15      0.18      9000
weighted avg       1.00      0.59      0.74      9000

F1-score is computed based on macro


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#if doing 3 label categories
predictions_mp, eval_mp = model_mp.predict(test_preprocessed, blanks)

36/36 [==============================] - 1s 13ms/step - loss: 3.8930 - accuracy: 0.5667
              precision    recall  f1-score   support

         0.0       1.00      0.57      0.72      9000
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00         0
         3.0       0.00      0.00      0.00         0
         4.0       0.00      0.00      0.00         0

    accuracy                           0.57      9000
   macro avg       0.20      0.11      0.14      9000
weighted avg       1.00      0.57      0.72      9000

F1-score is computed based on macro


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#if doing 3 label categories
label_dict = {
    '000' : 0,
    '100' : 1,
    '200' : 2,
    '300' : 3,
    '010' : 4,
    '020' : 5,
    '030' : 6,
    '001' : 7,
    '002' : 8,
    '003' : 9,
    '004' : 10,
    '110' : 11,
    '210' : 12,
    '310' : 13,
    '120' : 14,
    '220' : 15,
    '320' : 16,
    '130' : 17,
    '230' : 18,
    '330' : 19,
    '101' : 20,
    '201' : 21,
    '301' : 22,
    '102' : 23,
    '202' : 24,
    '302' : 25,
    '103' : 26,
    '203' : 27,
    '303' : 28,
}

In [ ]:
#if doing 3 label categories
predictions = {}
predictions['y_pred']=[]
for b,a,p in zip(predictions_ub['y_pred'],predictions_ma['y_pred'],predictions_mp['y_pred']):
    try:
        temp = str(b)+str(a)+str(p)
        predictions['y_pred'].append(label_dict[temp])
    except:
        predictions['y_pred'].append(29)

NameError: ignored

Count the number of predictions of each labels

In [ ]:
nums = Counter(predictions['y_pred']).values()
keys = Counter(predictions['y_pred']).keys()
print(keys)
print(nums)

dict_keys([24, 12, 17, 15, 27, 14, 9, 8, 10, 16, 0, 4, 13, 28, 11, 20, 23, 3, 25, 5, 18, 2, 19, 7, 21, 22, 1, 26, 6])
dict_values([87, 40, 40, 44, 130, 31, 112, 248, 64, 69, 105, 53, 76, 49, 30, 38, 71, 40, 49, 162, 27, 20, 33, 35, 40, 16, 36, 27, 28])


In [ ]:
submission = pd.read_csv('/content/drive/Shareddrives/Hackathon_Week11_Smart_Maintenance/sample_submission.csv')
submission

,filename,code
0,test_data0.csv,0.0
1,test_data1.csv,16.0
2,test_data2.csv,13.0
3,test_data3.csv,NaN
4,test_data4.csv,NaN
...,...,...
1795,test_data1795.csv,NaN
1796,test_data1796.csv,NaN
1797,test_data1797.csv,NaN
1798,test_data1798.csv,NaN


In [ ]:
for i in tqdm(range(1800)):
    filename = file_list[i]
    label = predictions['y_pred'][i]
    submission.loc[submission['filename']==filename, 'code'] = label
submission['code'] = submission['code'].apply(lambda x: int(x))
submission


100%|██████████| 1800/1800 [00:00<00:00, 1884.23it/s]


,filename,code
0,test_data0.csv,0
1,test_data1.csv,5
2,test_data2.csv,13
3,test_data3.csv,2
4,test_data4.csv,27
...,...,...
1795,test_data1795.csv,0
1796,test_data1796.csv,0
1797,test_data1797.csv,5
1798,test_data1798.csv,8


In [ ]:
submission.to_csv('/content/drive/MyDrive/week11_sub_eeg_weiner_600and1200.csv', index=False)